#Imports & Environment

In [ ]:
# Imports
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# Load environment 
load_dotenv()
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "mobilebank")
DB_USER = os.getenv("DB_USER", "postgres")
DB_PASSWORD = os.getenv("DB_PASSWORD", "8075")


#Connect to PostgreSQL

In [2]:
# Connect to PostgreSQL
engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

print("✔ Connected to PostgreSQL")


✔ Connected to PostgreSQL


#Load CSV

In [3]:
# Load final-review.csv
df = pd.read_csv(r"C:\Users\user\Desktop\Project\Mobile-Banking\data\processed\reviews_with_sentiment.csv")
print(f"Loaded {len(df)} reviews")
df.head()


Loaded 960 reviews


,review_id,review_text,rating,review_date,bank_code,bank_name,user_name,thumbs_up,reply_content,source,clean_review,compound,sentiment,clean_text
0,1,Very disappointing application. it's getting w...,1,2025-10-31 19:25:05,DASHEN,Dashen Bank,A.Senay ZENEBE,0,NaN,PlayStore,very disappointing application. it's getting w...,-0.7956,Negative,very disappointing application. it's getting w...
1,2,"Banking made simple, smart, and safe.",5,2025-10-28 15:35:23,DASHEN,Dashen Bank,Bilen Atinafe,0,NaN,PlayStore,"banking made simple, smart, and safe.",0.6808,Positive,"banking made simple, smart, and safe."
2,3,It is the best of all i liked it i used it alm...,5,2025-10-27 19:37:55,DASHEN,Dashen Bank,Guta Gundo,0,NaN,PlayStore,it is the best of all i liked it i used it alm...,0.8519,Positive,it is the best of all i liked it i used it alm...
3,4,gngu,5,2025-10-21 00:31:58,DASHEN,Dashen Bank,Yalelet Ahmed,0,NaN,PlayStore,gngu,0.0000,Neutral,gngu
4,5,"The app is very good , but it does not tell th...",4,2025-09-09 22:24:44,DASHEN,Dashen Bank,Esrael Worku,0,NaN,PlayStore,"the app is very good , but it does not tell th...",0.7504,Positive,"the app is very good , but it does not tell th..."


#Create Tables

In [7]:
try:
    engine = create_engine(
        f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    )
    connection = engine.connect()
    print("✔ Connected to PostgreSQL")
except OperationalError as e:
    print("❌ Connection failed. Check your PostgreSQL credentials and server status.")
    print(e)
    exit()

NameError: name 'OperationalError' is not defined

#: Insert Banks

In [ ]:
# Insert banks
banks = df["bank"].unique()

with engine.begin() as conn:  # Auto commit
    for bank in banks:
        conn.execute(text("""
        INSERT INTO banks (bank_name, app_name)
        VALUES (:bank_name, :app_name)
        ON CONFLICT (bank_name) DO NOTHING;
        """), {"bank_name": bank, "app_name": f"{bank} Mobile Banking App"})

    # Fetch bank_id mapping
    result = conn.execute(text("SELECT bank_id, bank_name FROM banks;"))
    bank_map = {row["bank_name"]: row["bank_id"] for row in result}

print(f"✔ Inserted {len(bank_map)} banks")


#Insert Reviews

In [ ]:
# Insert reviews
with engine.begin() as conn:
    for _, row in df.iterrows():
        conn.execute(text("""
        INSERT INTO reviews 
        (bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, source)
        VALUES (:bank_id, :review_text, :rating, :review_date, :sentiment_label, :sentiment_score, :source)
        """), {
            "bank_id": bank_map[row["bank"]],
            "review_text": row["review_text"],
            "rating": row.get("rating", None),
            "review_date": row.get("review_date", None),
            "sentiment_label": row.get("sentiment_label", None),
            "sentiment_score": row.get("sentiment_score", None),
            "source": "Google Play"
        })

print("✔ Inserted all reviews")
